In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import urllib.request
from selenium import webdriver
import pandas as pd
import numpy as np

leagues = ["Major_League_Baseball", "National_Basketball_Association", 
           "National_Football_League", "National_Hockey_League","Major_League_Soccer"]

In [2]:
for league in leagues:
    print("https://en.wikipedia.org/wiki/" + league)

https://en.wikipedia.org/wiki/Major_League_Baseball
https://en.wikipedia.org/wiki/National_Basketball_Association
https://en.wikipedia.org/wiki/National_Football_League
https://en.wikipedia.org/wiki/National_Hockey_League
https://en.wikipedia.org/wiki/Major_League_Soccer


In [3]:
def getLeagueAndCountry(league):
    joiner = []
    title = ''.join(c for c in league if c.isupper())
    URL = "https://en.wikipedia.org/wiki/" + league
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    table = soup.find('table', attrs = {'class': 'wikitable'})
    rows = table.findAll('tr')
    x = 2
    
    for row in rows:
        findB = row.find('b')
        try:
            findC = row.findAll('a')[1].text
        except:
            findC = ""
        x+=1
        if findB != None:
            if findB.text == findC:
                findC = row.findAll('a')[2].text
                
            joiner.append([findB.text, findC, title])
            
    time.sleep(10)
    
    return joiner;


In [4]:
def getGames(teamName):
    words = []
    img_src = []
    w = ""
    for c in range(len(teamName)):
        if teamName[c] != " ":
            w = w + teamName[c]
        else:
            words.append(w)
            w = ""
        if c == len(teamName) - 1:
            words.append(w)


    urlpage = 'https://www.google.com/search?q='
    for k in range(len(words)):
        urlpage = urlpage + words[k].lower()
        if k + 1 < len(words):
            urlpage = urlpage + "+"

    urlpage = urlpage + '&oq='

    for k in range(len(words)):
        urlpage = urlpage + words[k].lower()
        if k + 1 < len(words):
            urlpage = urlpage + "+"
    try:
        urlpage = urlpage + '&aqs=chrome.0.35i39j69i57j0l4.1669j1j4&sourceid=chrome&ie=UTF-8'
        driver = webdriver.Firefox()
        driver.get(urlpage)
        driver.execute_script("document.getElementsByClassName('LOaav')[0].click();")
        time.sleep(15)
        results = driver.find_elements_by_class_name("imspo_mt__tr")
        results2 = driver.find_elements_by_class_name("imspo_mt__cmd")
        data = []
        bo = False;
        x = 0
        img_src = []
        img = ""
        # loop over results
        for result in results:
            if (bo == False):
                x = 1
                bo = True
            else:
                x = 2
                bo = False
            product_name = result.text
            try:
                link = result.find_element_by_class_name('imspo_mt__t-sc').text
            except:
                link = "TBD"
            link3 = result.find_element_by_class_name('imso_btl__mh-logo')
            src = link3.get_attribute("src")
            img_src.append(src)
            link2 = result.find_element_by_class_name('ellipsisize').text
            # append dict to array
            data.append(["Team" + str(x),  link2 , link])
            
        unique,pos = np.unique(img_src,return_inverse=True) #Finds all unique elements and their positions
        counts = np.bincount(pos)                     #Count the number of each unique element
        maxpos = counts.argmax() 
        img = unique[maxpos]

        dates = []
        
        for result in results2:
            link = result.text
            if link != '':
                dates.append(link)
            
        results3 = driver.find_elements_by_class_name("imspo_mt__ns-pm-s")
        for result in results3:
            if (result.text != ""):
                dates.append(result.text.replace("\n", " "))

        data2 = []
        x = 0
        stall = False
        for k in range(len(data)):
            if (stall):
                stall = False
            else:
                try:
                    data2.append([data[k][1], data[k][2], data[k+1][1], data[k+1][2], dates[x]])
                    x+=1
                    stall = True
                except: 
                    break
        driver.quit()
        
        if len(data2) == 0:
            return ["", "No games found"]
        return [img, data2]
    
    except:
        try:
            driver.quit()
        except:
            print()
        print(teamName)
        return ["", "No games found"]

def index_containing_substring(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1

In [6]:
dArray = []
k = 0
o = 0
mapNG = []
game_arrays = []
gamesArr = []
for league in leagues:
    names = getLeagueAndCountry(league)
    for name in names:
        games = getGames(name[0])
        dArray.append({'id' : k, 'name' : name[0], 'country' : name[1], 'league' : name[2], 'icon' : games[0]})
        k += 1
        mapNG.append(name[0])
        for game in games[1]:
            gamesArr.append([game, name[2]])
    

Baltimore Orioles
Boston Red Sox
New York Yankees
Tampa Bay Rays
Toronto Blue Jays
Chicago White Sox
Cleveland Indians
Detroit Tigers
Kansas City Royals
Minnesota Twins
Houston Astros
Los Angeles Angels
Oakland Athletics
Seattle Mariners
Texas Rangers
Atlanta Braves
Miami Marlins
New York Mets
Philadelphia Phillies
Washington Nationals
Chicago Cubs
Cincinnati Reds
Milwaukee Brewers
Pittsburgh Pirates
St. Louis Cardinals
Arizona Diamondbacks
Colorado Rockies
Los Angeles Dodgers
San Diego Padres
San Francisco Giants
Atlanta United FC
Chicago Fire
FC Cincinnati
Columbus Crew SC
D.C. United
Inter Miami CF
Montreal Impact
New England Revolution
New York City FC
New York Red Bulls
Orlando City SC
Philadelphia Union
Toronto FC
Colorado Rapids
FC Dallas
Houston Dynamo
LA Galaxy
Los Angeles FC
Minnesota United FC
Nashville SC
Portland Timbers
Real Salt Lake
San Jose Earthquakes
Seattle Sounders FC
Sporting Kansas City
Vancouver Whitecaps FC


In [7]:
with open('team.txt', 'w') as f:
    for item in dArray:
        f.write("%s,\n" % item)



print(dArray)
print(mapNG)
print(gamesArr)

[{'id': 0, 'name': 'Baltimore Orioles', 'country': 'Baltimore, Maryland', 'league': 'MLB', 'icon': ''}, {'id': 1, 'name': 'Boston Red Sox', 'country': 'Boston, Massachusetts', 'league': 'MLB', 'icon': ''}, {'id': 2, 'name': 'New York Yankees', 'country': 'New York City, New York', 'league': 'MLB', 'icon': ''}, {'id': 3, 'name': 'Tampa Bay Rays', 'country': 'St. Petersburg, Florida', 'league': 'MLB', 'icon': ''}, {'id': 4, 'name': 'Toronto Blue Jays', 'country': 'Toronto, Ontario', 'league': 'MLB', 'icon': ''}, {'id': 5, 'name': 'Chicago White Sox', 'country': 'Chicago, Illinois', 'league': 'MLB', 'icon': ''}, {'id': 6, 'name': 'Cleveland Indians', 'country': 'Cleveland, Ohio', 'league': 'MLB', 'icon': ''}, {'id': 7, 'name': 'Detroit Tigers', 'country': 'Detroit, Michigan', 'league': 'MLB', 'icon': ''}, {'id': 8, 'name': 'Kansas City Royals', 'country': 'Kansas City, Missouri', 'league': 'MLB', 'icon': ''}, {'id': 9, 'name': 'Minnesota Twins', 'country': 'Minneapolis, Minnesota', 'leagu

In [15]:
gArry2 = []
for game in gamesArr:
    if len(game[0]) > 2:
        gArry2.append([game[0][0], game[0][1], game[0][2], game[0][3], game[0][4], game[1]])
print(gArry2)
print(len(gArry2))

[['Bucks', '105', 'Celtics', '116', '10/30', 'NBA'], ['Knicks', '102', 'Celtics', '104', '11/1', 'NBA'], ['Celtics', '119', 'Cavaliers', '113', '11/5', 'NBA'], ['Celtics', '108', 'Hornets', '87', '11/7', 'NBA'], ['Celtics', '135', 'Spurs', '115', '11/9', 'NBA'], ['Mavericks', '106', 'Celtics', '116', '11/11', 'NBA'], ['Wizards', '133', 'Celtics', '140', '11/13', 'NBA'], ['Celtics', '105', 'Warriors', '100', '11/15', 'NBA'], ['Celtics', '99', 'Kings', '100', '11/17', 'NBA'], ['Celtics', '99', 'Suns', '85', '11/18', 'NBA'], ['Celtics', '104', 'Clippers', '107', '11/20', 'NBA'], ['Celtics', '92', 'Nuggets', '96', '11/22', 'NBA'], ['Kings', '102', 'Celtics', '103', '11/25', 'NBA'], ['Nets', '110', 'Celtics', '121', '11/27', 'NBA'], ['Celtics', '107', 'Nets', '112', 'Fri, 11/29', 'NBA'], ['Celtics', '113', 'Knicks', '104', 'Sun, 12/1', 'NBA'], ['Heat', '93', 'Celtics', '112', 'Wed, 12/4', 'NBA'], ['Nuggets', '95', 'Celtics', '108', 'Fri, 12/6', 'NBA'], ['Cavaliers', '88', 'Celtics', '110', 

In [16]:
game_arrays = []
o = 0
unique_games = np.unique(gArry2, axis=0)
for game in unique_games:
    #indices = [i for i, s in enumerate(mapNG) if game[0] in s]
    #indices2 = [i for i, s in enumerate(mapNG) if game[2] in s]
    #id1 = mapNG[indices[0]]
    #id2 = mapNG[indices[0]]
    try:
        id1 = index_containing_substring(mapNG, game[0])
        id2 = index_containing_substring(mapNG, game[2])
        print(game, mapNG[id1], mapNG[id2])
        game_arrays.append({"id" : o, "home_team" : id1, "away_team" : id2, "league" : "NBA", 
                            "home_score" : game[1], "away_score" : game[3], "date" : game[4]})
        o+=1
    except:
        print("Nothing found")

print(len(game_arrays))

['49ers' '17' 'Ravens' '20' 'Sun, 12/1' 'NFL'] San Francisco 49ers Baltimore Ravens
['49ers' '20' 'Rams' '7' '10/13' 'NFL'] San Francisco 49ers Los Angeles Rams
['49ers' '24' 'Broncos' '15' '8/19/19' 'NFL'] San Francisco 49ers Denver Broncos
['49ers' '27' 'Chiefs' '17' '8/24/19' 'NFL'] San Francisco 49ers Kansas City Chiefs
['49ers' '28' 'Cardinals' '25' '10/31' 'NFL'] San Francisco 49ers St. Louis Cardinals
['49ers' '31' 'Buccaneers' '17' '9/8/19' 'NFL'] San Francisco 49ers Tampa Bay Buccaneers
['49ers' '41' 'Bengals' '17' '9/15' 'NFL'] San Francisco 49ers Cincinnati Bengals
['49ers' '48' 'Saints' '46' 'Sun, 12/8' 'NFL'] San Francisco 49ers New Orleans Saints
['49ers' '9' 'Redskins' '0' '10/20' 'NFL'] San Francisco 49ers Washington Redskins
['49ers' 'TBD' 'Seahawks' 'TBD' '12/29 4:25 PM' 'NFL'] San Francisco 49ers Seattle Seahawks
['76ers' '101' 'Knicks' '95' 'Fri, 11/29' 'NBA'] Philadelphia 76ers New York Knicks
['76ers' '104' 'Jazz' '106' '11/6' 'NBA'] Philadelphia 76ers Utah Jazz
[

['Blues' 'TBD' 'Oilers' 'TBD' '1/31 9:00 PM' 'NHL'] St. Louis Blues Edmonton Oilers
['Blues' 'TBD' 'Panthers' 'TBD' '3/21/20 7:00 PM' 'NHL'] St. Louis Blues Carolina Panthers
['Blues' 'TBD' 'Predators' 'TBD' '2/16 6:00 PM' 'NHL'] St. Louis Blues Nashville Predators
['Blues' 'TBD' 'Rangers' 'TBD' '3/3 7:00 PM' 'NHL'] St. Louis Blues Texas Rangers
['Blues' 'TBD' 'Sharks' 'TBD' 'Sat, 12/21 10:30 PM' 'NHL'] St. Louis Blues San Jose Sharks
['Blues' 'TBD' 'Stars' 'TBD' '2/21 8:30 PM' 'NHL'] St. Louis Blues Dallas Stars
['Blues' 'TBD' 'Wild' 'TBD' '2/23 7:30 PM' 'NHL'] St. Louis Blues Minnesota Wild
['Broncos' '13' 'Colts' '15' '10/27' 'NFL'] Denver Broncos Indianapolis Colts
['Broncos' '14' 'Falcons' '10' '8/1/19' 'NFL'] Denver Broncos Atlanta Falcons
['Broncos' '14' 'Seahawks' '22' '8/8/19' 'NFL'] Denver Broncos Seattle Seahawks
['Broncos' '16' 'Packers' '27' '9/22' 'NFL'] Denver Broncos Green Bay Packers
['Broncos' '16' 'Raiders' '24' '9/9/19' 'NFL'] Denver Broncos Oakland Raiders
['Bronco

['Capitals' 'TBD' 'Golden Knights' 'TBD' '2/17 6:00 PM' 'NHL'] Washington Capitals Vegas Golden Knights
['Capitals' 'TBD' 'Hurricanes' 'TBD' '1/3 7:30 PM' 'NHL'] Washington Capitals Carolina Hurricanes
['Capitals' 'TBD' 'Hurricanes' 'TBD' '12/28 7:00 PM' 'NHL'] Washington Capitals Carolina Hurricanes
['Capitals' 'TBD' 'Islanders' 'TBD' '1/18 1:00 PM' 'NHL'] Washington Capitals New York Islanders
['Capitals' 'TBD' 'Jets' 'TBD' '2/27 8:00 PM' 'NHL'] Washington Capitals New York Jets
['Capitals' 'TBD' 'Lightning' 'TBD' 'Sat, 12/14 7:00 PM' 'NHL'] Washington Capitals Tampa Bay Lightning
['Capitals' 'TBD' 'Panthers' 'TBD' '4/4/20 7:00 PM' 'NHL'] Washington Capitals Carolina Panthers
['Capitals' 'TBD' 'Penguins' 'TBD' '3/22/20 12:00 PM' 'NHL'] Washington Capitals Pittsburgh Penguins
['Capitals' 'TBD' 'Penguins' 'TBD' '3/7 1:00 PM' 'NHL'] Washington Capitals Pittsburgh Penguins
['Capitals' 'TBD' 'Rangers' 'TBD' '3/5 7:00 PM' 'NHL'] Washington Capitals Texas Rangers
['Capitals' 'TBD' 'Red Wing

['Devils' 'TBD' 'Coyotes' 'TBD' 'Sat, 12/14 8:00 PM' 'NHL'] New Jersey Devils Arizona Coyotes
['Devils' 'TBD' 'Ducks' 'TBD' '3/1 8:00 PM' 'NHL'] New Jersey Devils Anaheim Ducks
['Devils' 'TBD' 'Flyers' 'TBD' '2/6 7:00 PM' 'NHL'] New Jersey Devils Philadelphia Flyers
['Devils' 'TBD' 'Golden Knights' 'TBD' '3/3 10:00 PM' 'NHL'] New Jersey Devils Vegas Golden Knights
['Devils' 'TBD' 'Hurricanes' 'TBD' '2/14 7:30 PM' 'NHL'] New Jersey Devils Carolina Hurricanes
['Devils' 'TBD' 'Islanders' 'TBD' '1/2 7:00 PM' 'NHL'] New Jersey Devils New York Islanders
['Devils' 'TBD' 'Islanders' 'TBD' '4/4/20 7:00 PM' 'NHL'] New Jersey Devils New York Islanders
['Devils' 'TBD' 'Kings' 'TBD' '2/29 4:00 PM' 'NHL'] New Jersey Devils Sacramento Kings
['Devils' 'TBD' 'Lightning' 'TBD' '3/15/20 5:00 PM' 'NHL'] New Jersey Devils Tampa Bay Lightning
['Devils' 'TBD' 'Maple Leafs' 'TBD' '1/14 7:00 PM' 'NHL'] New Jersey Devils Toronto Maple Leafs
['Devils' 'TBD' 'Maple Leafs' 'TBD' '3/17/20 7:00 PM' 'NHL'] New Jersey

['Hawks' '103' 'Pistons' '128' '11/22' 'NBA'] Atlanta Hawks Detroit Pistons
['Hawks' '104' 'Pacers' '105' 'Fri, 11/29' 'NBA'] Atlanta Hawks Indiana Pacers
['Hawks' '111' 'Rockets' '158' 'Sat, 11/30' 'NBA'] Atlanta Hawks Houston Rockets
['Hawks' '112' 'Suns' '128' '11/14' 'NBA'] Atlanta Hawks Phoenix Suns
['Hawks' '113' 'Trail Blazers' '124' '11/10' 'NBA'] Atlanta Hawks Portland Trail Blazers
['Hawks' '121' 'Heat' '135' 'Tue, 12/10' 'NBA'] Atlanta Hawks Miami Heat
['Hawks' '122' 'Hornets' '107' 'Sun, 12/8' 'NBA'] Atlanta Hawks Charlotte Hornets
['Hawks' '125' 'Nuggets' '121' '11/12' 'NBA'] Atlanta Hawks Denver Nuggets
['Hawks' '97' 'Heat' '112' '10/29' 'NBA'] Atlanta Hawks Miami Heat
['Hawks' 'TBD' '76ers' 'TBD' '2/24 7:00 PM' 'NBA'] Atlanta Hawks Philadelphia 76ers
['Hawks' 'TBD' '76ers' 'TBD' '3/21/20 7:00 PM' 'NBA'] Atlanta Hawks Philadelphia 76ers
['Hawks' 'TBD' 'Bucks' 'TBD' '4/12/20 7:00 PM' 'NBA'] Atlanta Hawks Milwaukee Bucks
['Hawks' 'TBD' 'Bulls' 'TBD' '12/28 8:00 PM' 'NBA'] A

...

['Lions' '16' 'Browns' '20' '8/29/19' 'NFL'] Detroit Lions Cleveland Browns
['Lions' '16' 'Redskins' '19' '11/24' 'NFL'] Detroit Lions Washington Redskins
['Lions' '22' 'Packers' '23' '10/14' 'NFL'] Detroit Lions Green Bay Packers
['Lions' '23' 'Texans' '30' '8/17/19' 'NFL'] Detroit Lions Houston Texans
['Lions' '24' 'Raiders' '31' '11/3' 'NFL'] Detroit Lions Oakland Raiders
['Lions' '27' 'Cardinals' '27' '9/8/19' 'NFL'] Detroit Lions St. Louis Cardinals
['Lions' '27' 'Eagles' '24' '9/22' 'NFL'] Detroit Lions Philadelphia Eagles
['Lions' '7' 'Vikings' '20' 'Sun, 12/8' 'NFL'] Detroit Lions Minnesota Vikings
['Lions' 'TBD' 'Broncos' 'TBD' 'Sun, 12/22 4:05 PM' 'NFL'] Detroit Lions Denver Broncos
['Magic' '101' 'Bucks' '110' 'Mon, 12/9' 'NBA'] Orlando Magic Milwaukee Bucks
['Magic' '106' 'Mavericks' '107' '11/6' 'NBA'] Orlando Magic Dallas Mavericks
['Magic' '106' 'Pacers' '111' '11/23' 'NBA'] Orlando Magic Indiana Pacers
['Magic' '116' 'Cavaliers' '104' '11/27' 'NBA'] Orlando Magic Clevel

['Packers' '31' 'Giants' '13' 'Sun, 12/1' 'NFL'] Green Bay Packers San Francisco Giants
['Packers' '34' 'Cowboys' '24' '10/6' 'NFL'] Green Bay Packers Dallas Cowboys
['Packers' '8' '49ers' '37' '11/24' 'NFL'] Green Bay Packers San Francisco 49ers
['Packers' 'TBD' 'Lions' 'TBD' '12/29 1:00 PM' 'NFL'] Green Bay Packers Detroit Lions
['Packers' 'TBD' 'Vikings' 'TBD' 'Mon, 12/23 8:15 PM' 'NFL'] Green Bay Packers Minnesota Vikings
['Panthers' '1' 'Islanders' '2' '11/9' 'NHL'] Carolina Panthers New York Islanders
['Panthers' '13' '49ers' '51' '10/27' 'NFL'] Carolina Panthers San Francisco 49ers
['Panthers' '16' 'Packers' '24' '11/10' 'NFL'] Carolina Panthers Green Bay Packers
['Panthers' '16' 'Texans' '10' '9/29' 'NFL'] Carolina Panthers Houston Texans
['Panthers' '2' 'Canucks' '7' '10/28' 'NHL'] Carolina Panthers Vancouver Canucks
['Panthers' '2' 'Hurricanes' '4' '11/23' 'NHL'] Carolina Panthers Carolina Hurricanes
['Panthers' '20' 'Falcons' '40' 'Sun, 12/8' 'NFL'] Carolina Panthers Atlanta

['Rams' '34' 'Cardinals' '7' 'Sun, 12/1' 'NFL'] Los Angeles Rams St. Louis Cardinals
['Rams' '37' 'Falcons' '10' '10/20' 'NFL'] Los Angeles Rams Atlanta Falcons
['Rams' 'TBD' '49ers' 'TBD' 'Sat, 12/21 8:15 PM' 'NFL'] Los Angeles Rams San Francisco 49ers
['Rams' 'TBD' 'Cowboys' 'TBD' 'Sun, 12/15 4:25 PM' 'NFL'] Los Angeles Rams Dallas Cowboys
['Rangers' '1' 'Kings' '3' 'Tue, 12/10' 'NHL'] Texas Rangers Sacramento Kings
['Rangers' '1' 'Senators' '4' '11/22' 'NHL'] Texas Rangers Ottawa Senators
['Rangers' '2' 'Bruins' '3' 'Fri, 11/29' 'NHL'] Texas Rangers Boston Bruins
['Rangers' '2' 'Predators' '1' '11/2' 'NHL'] Texas Rangers Nashville Predators
['Rangers' '3' 'Blue Jackets' '2' 'Thu, 12/5' 'NHL'] Texas Rangers Columbus Blue Jackets
['Rangers' '3' 'Lightning' '9' '11/14' 'NHL'] Texas Rangers Tampa Bay Lightning
['Rangers' '3' 'Panthers' '4' '11/16' 'NHL'] Texas Rangers Carolina Panthers
['Rangers' '4' 'Devils' '0' 'Sat, 11/30' 'NHL'] Texas Rangers New Jersey Devils
['Rangers' '4' 'Hurric

['Senators' '1' 'Flames' '3' 'Sat, 11/30' 'NHL'] Ottawa Senators Calgary Flames
['Senators' '1' 'Islanders' '4' '11/5' 'NHL'] Ottawa Senators New York Islanders
['Senators' '1' 'Stars' '2' '10/21' 'NHL'] Ottawa Senators Dallas Stars
['Senators' '2' 'Bruins' '5' '11/2' 'NHL'] Ottawa Senators Boston Bruins
['Senators' '2' 'Canadiens' '1' '11/20' 'NHL'] Ottawa Senators Montreal Canadiens
['Senators' '2' 'Canadiens' '3' 'Yesterday' 'NHL'] Ottawa Senators Montreal Canadiens
['Senators' '2' 'Canucks' '5' 'Tue, 12/3' 'NHL'] Ottawa Senators Vancouver Canucks
['Senators' '2' 'Hurricanes' '8' '11/11' 'NHL'] Ottawa Senators Carolina Hurricanes
['Senators' '2' 'Sabres' '4' '11/16' 'NHL'] Ottawa Senators Buffalo Sabres
['Senators' '2' 'Wild' '7' 'Fri, 11/29' 'NHL'] Ottawa Senators Minnesota Wild
['Senators' '3' 'Flyers' '4' 'Sat, 12/7' 'NHL'] Ottawa Senators Philadelphia Flyers
['Senators' '4' 'Devils' '2' '11/13' 'NHL'] Ottawa Senators New Jersey Devils
['Senators' '4' 'Red Wings' '3' '11/19' 'NHL

['Thunder' 'TBD' 'Suns' 'TBD' '1/31 9:00 PM' 'NBA'] Oklahoma City Thunder Phoenix Suns
['Thunder' 'TBD' 'Timberwolves' 'TBD' '1/13 8:00 PM' 'NBA'] Oklahoma City Thunder Minnesota Timberwolves
['Thunder' 'TBD' 'Timberwolves' 'TBD' '1/25 8:00 PM' 'NBA'] Oklahoma City Thunder Minnesota Timberwolves
['Thunder' 'TBD' 'Warriors' 'TBD' '3/28/20 8:30 PM' 'NBA'] Oklahoma City Thunder Golden State Warriors
['Thunder' 'TBD' 'Wizards' 'TBD' '3/15/20 6:00 PM' 'NBA'] Oklahoma City Thunder Washington Wizards
['Timberwolves' '109' 'Suns' '125' 'Mon, 12/9' 'NBA'] Minnesota Timberwolves Phoenix Suns
['Timberwolves' '112' 'Jazz' '102' '11/18' 'NBA'] Minnesota Timberwolves Utah Jazz
['Timberwolves' '113' 'Spurs' '101' '11/27' 'NBA'] Minnesota Timberwolves San Antonio Spurs
['Timberwolves' '114' 'Mavericks' '121' 'Wed, 12/4' 'NBA'] Minnesota Timberwolves Dallas Mavericks
['Timberwolves' '120' 'Pistons' '114' '11/11' 'NBA'] Minnesota Timberwolves Detroit Pistons
['Timberwolves' '121' 'Grizzlies' '137' '11/6

In [17]:
with open('game_file.txt', 'w') as f:
    for item in game_arrays:
        f.write("%s,\n" % item)

In [5]:
games = getGames('Dallas Mavericks')

In [6]:
print(games)

[{'Team1': 'Chargers 13', 'Team2': 'Cardinals 17', 'Date': '8/8/19'}, {'Team1': 'Raiders 33', 'Team2': 'Cardinals 26', 'Date': '8/15/19'}, {'Team1': 'Cardinals 9', 'Team2': 'Vikings 20', 'Date': '8/24/19'}, {'Team1': 'Cardinals 7', 'Team2': 'Broncos 20', 'Date': '8/29/19'}, {'Team1': 'Lions 27', 'Team2': 'Cardinals 27', 'Date': '9/8/19'}, {'Team1': 'Cardinals 17', 'Team2': 'Ravens 23', 'Date': '9/15'}, {'Team1': 'Panthers 38', 'Team2': 'Cardinals 20', 'Date': '9/22'}, {'Team1': 'Seahawks 27', 'Team2': 'Cardinals 10', 'Date': '9/29'}, {'Team1': 'Cardinals 26', 'Team2': 'Bengals 23', 'Date': '10/6'}, {'Team1': 'Falcons 33', 'Team2': 'Cardinals 34', 'Date': '10/13'}, {'Team1': 'Cardinals 27', 'Team2': 'Giants 21', 'Date': '10/20'}, {'Team1': 'Cardinals 9', 'Team2': 'Saints 31', 'Date': '10/27'}, {'Team1': '49ers 28', 'Team2': 'Cardinals 25', 'Date': '10/31'}, {'Team1': 'Cardinals 27', 'Team2': 'Buccaneers 30', 'Date': '11/10'}, {'Team1': 'Cardinals 26', 'Team2': '49ers 36', 'Date': '11/17